### Installing the required libraries

In [ ]:

pip install datasets transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Authenticating the huggng face to login
from huggingface_hub import login
login()

### Loading the dataset from the hugging face

In [4]:
from datasets import load_dataset, concatenate_datasets

domains = ['educational_institution']

datasets = [load_dataset("neulab/wiki_asp", domain, trust_remote_code=True)["train"] for domain in domains]
dataset = concatenate_datasets(datasets)

print(dataset)
print("Total samples:", len(dataset))


Dataset({
    features: ['exid', 'inputs', 'targets'],
    num_rows: 17634
})
Total samples: 17634


### Preprocessing the data

In [ ]:
from datasets import Dataset
import numpy as np

def _merge_inputs(x):
    # inputs can be a list of strings so joining them
    if isinstance(x, list):
        return " ".join([str(s) for s in x]).strip()
    return str(x).strip()

def _merge_targets(t):
    # targets can be the list of [aspect, summary] pairs OR strings
    out = []
    if isinstance(t, list):
        for item in t:
            if isinstance(item, (list, tuple)) and len(item) >= 2:
                out.append(str(item[1]))
            elif isinstance(item, str):
                out.append(item)
    else:
        out.append(str(t))
    return " ".join(out).strip()

def normalize_batch(batch):
    inputs_text  = [_merge_inputs(x) for x in batch["inputs"]]
    targets_text = [_merge_targets(x) for x in batch["targets"]]
    return {"inputs_text": inputs_text, "targets_text": targets_text}

normalized = dataset.map(normalize_batch, batched=True, desc="Normalising inputs/targets")

# basic empties check
num_empty_src = sum(1 for x in normalized["inputs_text"] if not x)
num_empty_tgt = sum(1 for x in normalized["targets_text"] if not x)
print(f"Empty inputs: {num_empty_src} | Empty targets: {num_empty_tgt}")

# quick length stats (characters)
src_lens = np.array([len(x) for x in normalized["inputs_text"]])
tgt_lens = np.array([len(x) for x in normalized["targets_text"]])
def q(a, p): 
    i = int(len(a) * p)
    return int(np.sort(a)[i])

print(
    f"Inputs chars: min={src_lens.min()} "
    f"p50={q(src_lens,0.5)} p90={q(src_lens,0.9)} max={src_lens.max()}"
)
print(
    f"Targets chars: min={tgt_lens.min()} "
    f"p50={q(tgt_lens,0.5)} p90={q(tgt_lens,0.9)} max={tgt_lens.max()}"
)

# one normalised example
i = 0
print("\nSample INPUT:\n", normalized[i]["inputs_text"][:400])
print("\nSample TARGET:\n", normalized[i]["targets_text"][:400])


Normalising inputs/targets:   0%|          | 0/17634 [00:00<?, ? examples/s]

Empty inputs: 0 | Empty targets: 783
Inputs chars: min=13 p50=44650 p90=209941 max=2198917
Targets chars: min=0 p50=1219 p90=4100 max=36039

Sample INPUT:
 < EOT > the church of st . peter at the castle , later known as st . peter in the west or le bailey , was granted to st . fridewide ' s priory in 1122 . ( fn . 956 ) the living was held in plurality with st . ebbe ' s from 1913 to 1926 and with st . aldate ' s 1927 - 8 . in 1928 the church and its property were used for the foundation of st . peter ' s hall , later st . peter ' s college , and fro

Sample TARGET:
 in 1320 , adam de brome was appointed rector of the church of st mary the virgin . along with the appointment , he was given the rectory house , st mary hall , on the high street . st mary hall was acquired by oriel college in 1326 . bedel hall , adjoining st mary ' s to the south , was given by bishop carpenter of worcester in 1455 . these two halls , along with st martin ' s hall , served as ann


In [ ]:
import re
from datasets import Dataset

# cleaning the text
RE_EOT = re.compile(r"\s*<\s*EOT\s*>\s*", re.IGNORECASE)
RE_FOOTNOTE = re.compile(r"\(\s*fn\s*\.\s*\d+\s*\)", re.IGNORECASE)
RE_SPACES = re.compile(r"\s+")

def tidy_punct(s: str) -> str:
    # normalising the spaces around punctuation
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)
    s = re.sub(r"([(\[])\\s+","\\1", s)
    s = re.sub(r"\s+([)\]])", r"\1", s)
    return s

def clean_str(s: str) -> str:
    s = RE_EOT.sub(" ", s)
    s = RE_FOOTNOTE.sub(" ", s)
    s = RE_SPACES.sub(" ", s).strip()
    s = tidy_punct(s)
    return s

def clean_batch(batch):
    src = [clean_str(x) for x in batch["inputs_text"]]
    tgt = [clean_str(x) for x in batch["targets_text"]]
    return {"inputs_text": src, "targets_text": tgt}

cleaned = normalized.map(clean_batch, batched=True, desc="Cleaning text")

# dropping the entries with empty target after cleaning
cleaned = cleaned.filter(lambda t: bool(t and t.strip()), input_columns=["targets_text"], desc="Drop empty targets")

# deduplicating it by source text (exact match)
seen = set()
keep_idx = []
for i, x in enumerate(cleaned["inputs_text"]):
    if x in seen:
        continue
    seen.add(x)
    keep_idx.append(i)

cleaned = cleaned.select(keep_idx)

# renaming to the final column names
cleaned = cleaned.rename_columns({"inputs_text": "source", "targets_text": "summary"})

print(cleaned)
print("Examples after cleaning:", len(cleaned))
print("Sample source:", cleaned[0]["source"][:300])
print("\nSample summary:", cleaned[0]["summary"][:300])


Cleaning text:   0%|          | 0/17634 [00:00<?, ? examples/s]

Drop empty targets:   0%|          | 0/17634 [00:00<?, ? examples/s]

Dataset({
    features: ['exid', 'inputs', 'targets', 'source', 'summary'],
    num_rows: 16813
})
Examples after cleaning: 16813
Sample source: the church of st. peter at the castle, later known as st. peter in the west or le bailey, was granted to st. fridewide ' s priory in 1122. the living was held in plurality with st. ebbe ' s from 1913 to 1926 and with st. aldate ' s 1927 - 8. in 1928 the church and its property were used for the foun

Sample summary: in 1320, adam de brome was appointed rector of the church of st mary the virgin. along with the appointment, he was given the rectory house, st mary hall, on the high street. st mary hall was acquired by oriel college in 1326. bedel hall, adjoining st mary ' s to the south, was given by bishop carpe


### Loading the base model

In [ ]:
from transformers import AutoTokenizer

# Picking the base model
BASE_CKPT = "t5-base"
tok = AutoTokenizer.from_pretrained(BASE_CKPT, use_fast=True)

def add_prefix(ex):
    ex["source_text"] = "summarize: " + ex["source"]
    ex["target_text"] = ex["summary"]
    return ex

with_prefix = cleaned.map(add_prefix, desc="Adding task prefix")

def add_lengths(ex):
    # getting the token lengths without truncation so we can filter
    ex["input_len"]  = len(tok.encode(ex["source_text"], truncation=False))
    ex["target_len"] = len(tok.encode(ex["target_text"], truncation=False))
    return ex

with_lens = with_prefix.map(add_lengths, desc="Measuring token lengths")

# Reasonable bounds for T5/BART fine-tuning
MIN_IN, MAX_IN = 32, 1024     # inputs between 32 and 1024 tokens
MIN_OUT, MAX_OUT = 8, 200     # targets between 8 and 200 tokens

filtered = with_lens.filter(
    lambda il, tl: (MIN_IN <= il <= MAX_IN) and (MIN_OUT <= tl <= MAX_OUT),
    input_columns=["input_len","target_len"],
    desc="Filtering by token length"
)

# Keeping only the final columns used for training 
cols_to_keep = ["source_text", "target_text"]
processed = filtered.remove_columns([c for c in filtered.column_names if c not in cols_to_keep])

print(processed)
print("Total examples after filtering:", len(processed))
print("\nSample pair:")
print("SOURCE:", processed[0]["source_text"][:220])
print("\nTARGET:", processed[0]["target_text"][:220])


Measuring token lengths:   0%|          | 0/16813 [00:00<?, ? examples/s]

Filtering by token length:   0%|          | 0/16813 [00:00<?, ? examples/s]

Dataset({
    features: ['source_text', 'target_text'],
    num_rows: 584
})
Total examples after filtering: 584

Sample pair:
SOURCE: summarize: support the friends of caerleon comprehensive school by joining our school lottery - all proceeds benefit the school. follow the link below: caerleon comprehensive school uses cookies to improve our website an

TARGET: according to a 2016 estyn inspection, all students leaving at key stage 4 left with a recognized qualification. department for education performance tables reveal caerleon comprehensive ' s success record, with an averag


In [ ]:
from datasets import DatasetDict

# current processed dataset is names as `processed`
print(processed)

# 80/10/10 split
tmp = processed.train_test_split(test_size=0.20, seed=42, shuffle=True)
val_test = tmp["test"].train_test_split(test_size=0.50, seed=42)

final_ds = DatasetDict({
    "train": tmp["train"],
    "validation": val_test["train"],
    "test": val_test["test"]
})

print(final_ds)
print("Counts:", {k: len(v) for k,v in final_ds.items()})

# saving it for reuse
save_dir = "data/wikiasp_educational_institution_t5_prepared"
final_ds.save_to_disk(save_dir)
print(f"Saved processed splits to: {save_dir}")


Dataset({
    features: ['source_text', 'target_text'],
    num_rows: 584
})
DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text'],
        num_rows: 467
    })
    validation: Dataset({
        features: ['source_text', 'target_text'],
        num_rows: 58
    })
    test: Dataset({
        features: ['source_text', 'target_text'],
        num_rows: 59
    })
})
Counts: {'train': 467, 'validation': 58, 'test': 59}


Saving the dataset (0/1 shards):   0%|          | 0/467 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/58 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/59 [00:00<?, ? examples/s]

Saved processed splits to: data/wikiasp_educational_institution_t5_prepared


In [ ]:
## Installing the required libraries
pip install transformers datasets peft accelerate evaluate rouge-score sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl.metadata (29 kB)
Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl (68.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip uninstall -y transformers peft accelerate tokenizers sentencepiece
!pip install --upgrade "transformers==4.44.2" "peft==0.11.1" "accelerate==0.34.2" "sentencepiece>=0.1.99" "evaluate>=0.4.2" "rouge-score>=0.1.2"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: peft 0.11.1
Uninstalling peft-0.11.1:
  Successfully uninstalled peft-0.11.1
Found existing installation: accelerate 0.34.2
Uninstalling accelerate-0.34.2:
  Successfully uninstalled accelerate-0.34.2
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached peft-0.11.1-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached peft-0.11.1-py3-none-any.whl (251 kB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
Using cached tokenizers-0.19.1-cp310-cp310-macosx_11_0_arm64.whl (2.4 MB)
Using cached sentencepiece-0.2.1-cp310-cp310-macosx_11_0_arm64.whl (1.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [peft]3/5 [transformers]


### Model Fine tuning 

In [ ]:
# Importing the required libraries
import os
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate
import numpy as np

In [8]:
data_dir = "data/wikiasp_educational_institution_t5_prepared"
ds = load_from_disk(data_dir)

BASE_CKPT = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(BASE_CKPT, use_fast=True)


/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
MAX_SRC_LEN = 512
MAX_TGT_LEN = 128

def tokenize_fn(batch):
    model_inputs = tokenizer(
        batch["source_text"],
        max_length=MAX_SRC_LEN,
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target_text"],
            max_length=MAX_TGT_LEN,
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenised = ds.map(tokenize_fn, batched=True, remove_columns=["source_text","target_text"])
print(tokenised)


Map:   0%|          | 0/467 [00:00<?, ? examples/s]

/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 467
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 58
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 59
    })
})


In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(BASE_CKPT)

# LoRA config (lightweight)
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # T5 attention proj names
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # decode
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True)
    # returning the mean scores
    return {k: round(v * 100, 2) for k, v in scores.items()}


In [13]:
from transformers import Seq2SeqTrainingArguments

out_dir = "runs/t5_base_lora_wikiasp_edu"

args = Seq2SeqTrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=8,
    lr_scheduler_type="linear",
    warmup_ratio=0.06,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=25,
    predict_with_generate=True,          # now valid
    generation_max_length=MAX_TGT_LEN,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
)


/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### Fine tuning the model

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/232 [00:00<?, ?it/s]

/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 4.6906, 'grad_norm': 0.5308341979980469, 'learning_rate': 0.00018990825688073394, 'epoch': 0.85}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 4.1148505210876465, 'eval_rouge1': 17.01, 'eval_rouge2': 3.91, 'eval_rougeL': 12.15, 'eval_rougeLsum': 12.27, 'eval_runtime': 66.0654, 'eval_samples_per_second': 0.878, 'eval_steps_per_second': 0.227, 'epoch': 0.99}


...

{'loss': 3.29, 'grad_norm': 0.4510098397731781, 'learning_rate': 6.422018348623854e-06, 'epoch': 7.69}


/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 3.0646309852600098, 'eval_rouge1': 18.7, 'eval_rouge2': 3.26, 'eval_rougeL': 15.27, 'eval_rougeLsum': 15.26, 'eval_runtime': 117.9324, 'eval_samples_per_second': 0.492, 'eval_steps_per_second': 0.127, 'epoch': 7.93}
{'train_runtime': 1470.1313, 'train_samples_per_second': 2.541, 'train_steps_per_second': 0.158, 'train_loss': 3.6029654782393883, 'epoch': 7.93}


TrainOutput(global_step=232, training_loss=3.6029654782393883, metrics={'train_runtime': 1470.1313, 'train_samples_per_second': 2.541, 'train_steps_per_second': 0.158, 'total_flos': 2249289612619776.0, 'train_loss': 3.6029654782393883, 'epoch': 7.931623931623932})

In [15]:
print(trainer.evaluate(tokenised["test"]))

def generate(text, max_new_tokens=128):
    enc = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, max_length=MAX_SRC_LEN)
    enc = {k: v.to(model.device) for k,v in enc.items()}
    out = model.generate(**enc, max_new_tokens=max_new_tokens, num_beams=4)
    return tokenizer.decode(out[0], skip_special_tokens=True)

sample_src = ds["test"][0]["source_text"]
print("\nSOURCE:\n", sample_src[:800])
print("\nSUMMARY:\n", generate(sample_src))


/opt/anaconda3/envs/textsum/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 3.085615396499634, 'eval_rouge1': 16.74, 'eval_rouge2': 2.74, 'eval_rougeL': 13.09, 'eval_rougeLsum': 13.14, 'eval_runtime': 123.3469, 'eval_samples_per_second': 0.478, 'eval_steps_per_second': 0.122, 'epoch': 7.931623931623932}

SOURCE:
 summarize: western mst magnet high school is located in louisville, kentucky. scholarships for students attending western mst magnet high school can be found in our scholarship database. if you are searching for scholarships for students attending a different high school in louisville, visit the louisville directory. <EOS>

SUMMARY:
 western mst magnet high school is located in louisville, kentucky.


In [16]:
import warnings
warnings.filterwarnings('ignore')

### Evaluating the model

In [17]:
test_metrics = trainer.evaluate(tokenised["test"])
print("Test metrics:", test_metrics)


  0%|          | 0/15 [00:00<?, ?it/s]

Test metrics: {'eval_loss': 3.085615396499634, 'eval_rouge1': 16.74, 'eval_rouge2': 2.74, 'eval_rougeL': 13.09, 'eval_rougeLsum': 13.14, 'eval_runtime': 105.3595, 'eval_samples_per_second': 0.56, 'eval_steps_per_second': 0.142, 'epoch': 7.931623931623932}


#### Saving the model

In [18]:
save_dir = "artifacts/t5_base_lora_wikiasp_edu"
model.save_pretrained(save_dir)         # saves LoRA adapter weights
tokenizer.save_pretrained(save_dir)
print("Saved adapter + tokenizer to:", save_dir)


Saved adapter + tokenizer to: artifacts/t5_base_lora_wikiasp_edu


In [19]:
from peft import PeftModel
from transformers import AutoModelForSeq2SeqLM

merged_dir = "artifacts/t5_base_merged_wikiasp_edu"

# reload base + adapter, then merge
base = AutoModelForSeq2SeqLM.from_pretrained(BASE_CKPT)
peft_loaded = PeftModel.from_pretrained(base, save_dir)

merged = peft_loaded.merge_and_unload()     # folds LoRA weights into the base
merged.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)

print("Saved merged full model to:", merged_dir)


Saved merged full model to: artifacts/t5_base_merged_wikiasp_edu


In [ ]:
sample = ds["test"][0]["source_text"]

# Using the merged model 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tok_m = AutoTokenizer.from_pretrained(merged_dir)
mdl_m = AutoModelForSeq2SeqLM.from_pretrained(merged_dir)

enc = tok_m("summarize: " + sample, return_tensors="pt", truncation=True, max_length=512)
out = mdl_m.generate(**enc, max_new_tokens=128, num_beams=4)
print("\nMERGED MODEL SUMMARY:\n", tok_m.decode(out[0], skip_special_tokens=True))

# Using the adapter + base (PEFT) 
from peft import PeftModel
tok_a = AutoTokenizer.from_pretrained(save_dir)
base = AutoModelForSeq2SeqLM.from_pretrained(BASE_CKPT)
mdl_a = PeftModel.from_pretrained(base, save_dir)

enc = tok_a("summarize: " + sample, return_tensors="pt", truncation=True, max_length=512)
out = mdl_a.generate(**enc, max_new_tokens=128, num_beams=4)
print("\nADAPTER MODEL SUMMARY:\n", tok_a.decode(out[0], skip_special_tokens=True))



MERGED MODEL SUMMARY:
 western mst magnet high school is located in louisville, kentucky.

ADAPTER MODEL SUMMARY:
 western mst magnet high school is located in louisville, kentucky.


In [24]:
!pip install flask transformers torch sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Web part Integration

In [34]:
!python app.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading model from artifacts/t5_base_merged_wikiasp_edu on cpu...
Model loaded successfully.
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://10.131.187.2:8000
Press CTRL+C to quit
 * Restarting with stat
Loading model from artifacts/t5_base_merged_wikiasp_edu on cpu...
Model loaded successfully.
 * Debugger is active!
 * Debugger PIN: 197-921-946
127.0.0.1 - - [19/Oct/2025 00:15:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:16:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:16:43] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/Oct/2025 00:16:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:17:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:17:37] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/Oct/2025 00:17:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:17:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:18:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - 